In [1]:
import numpy as np

import muspy
import json

import torch.nn as nn

In [12]:
NAME_MUSIC = 'essen'
DATA_TOKENS = f"data/songs_token_{NAME_MUSIC}.json"
MODELS = 'models_exp'
songs = []
with open(DATA_TOKENS, "r", encoding="utf-8") as f:
    songs = json.load(f)

print(len(songs))
print(len(songs[0]))

1000
41


In [3]:
import torch
import torch.nn as nn
import numpy as np

class SimpleMusicLSTM(nn.Module):

    def __init__(self, hidden_size=128, num_layers=2):
        super().__init__()


        # LSTM принимает (pitch, duration, rest) напрямую
        self.lstm = nn.LSTM(
            input_size=3,
            hidden_size=hidden_size, # размер скрытого состояния
            num_layers=num_layers,   # количество слоев LSTM
            batch_first=True,        # формат [batch, seq, features]
            dropout=0.2 if num_layers > 1 else 0  # dropout между слоями
        )

        # Полносвязный слой для предсказания
        self.fc = nn.Linear(hidden_size, 3)  # 3 выхода

        # Dropout для регуляризации
        self.dropout = nn.Dropout(0.2)

        # Сохраняем параметры
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, x):
        """
        Args:
            x: [batch_size, seq_length, 3]
               batch_size = количество последовательностей
               seq_length = длина последовательности (например, 20)
               3 = (pitch, duration, rest)

        Returns:
            prediction: [batch_size, 3] - следующая нота
        """
        #    LSTM обрабатывает всю последовательность
        #    Видит 20 нот, понимает паттерны
        lstm_out, _ = self.lstm(x)  # [batch, seq_len, hidden_size]

        last_output = lstm_out[:, -1, :]

        #  Применяем dropout (для регуляризации)
        last_output = self.dropout(last_output)

        #  Предсказываем следующую ноту
        prediction = self.fc(last_output)

        return prediction

    def init_hidden(self, batch_size, device='cpu'):
        """Инициализация скрытого состояния (для генерации)"""
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))


In [4]:


def prepare_data_simple(songs, seq_length=20):
    """
    Подготавливает данные для простой модели
    """
    X, y = [], []

    for song in songs:
        if len(song) > seq_length:
            # Разбиваем песню на последовательности
            for i in range(len(song) - seq_length):
                # Контекст: seq_length нот
                sequence = song[i:i + seq_length]
                #  следующая нота
                target = song[i + seq_length]

                X.append(sequence)
                y.append(target)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)

    print(f"Создано {len(X)} обучающих пар")
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    return X, y

def train_simple_model(model, X_train, y_train, epochs=30, batch_size=64, lr=0.001):
    """Обучает модель"""

    X_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_tensor = torch.tensor(y_train, dtype=torch.float32)

    dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True
    )

    # Оптимизатор и функция потерь
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # MSE loss для регрессии (предсказываем числа)
    criterion = nn.MSELoss()

    # Обучение
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0

        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()

            # Прямой проход
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)

            # Обратный проход
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

    print("Обучение завершено!")
    return model

def generate_music_simple(model, seed_sequence, length=200, temperature=0.5):
    """
    Генерирует музыку с помощью модели
    """
    model.eval()

    current_sequence = seed_sequence.copy()
    generated = []

    with torch.no_grad():
        for _ in range(length):
            # Берем последние 20 нот (или меньше если начало)
            if len(current_sequence) > 20:
                context = current_sequence[-20:]
            else:
                context = current_sequence

            # Добавляем padding если нужно
            if len(context) < 20:
                padding = [(0, 0, 0)] * (20 - len(context))
                context = padding + context

            # Конвертируем в тензор
            context_tensor = torch.tensor([context], dtype=torch.float32)

            # Предсказываем следующую ноту
            prediction = model(context_tensor)[0]

            # Добавляем немного случайности
            noise = torch.randn_like(prediction) * temperature
            predicted_note = prediction + noise

            # Округляем и ограничиваем значения
            pitch = int(torch.clamp(predicted_note[0], 0, 127).item())
            duration = int(torch.clamp(predicted_note[1], 1, 16).item())
            rest = int(torch.clamp(predicted_note[2], 0, 16).item())

            next_note = (pitch, duration, rest)

            current_sequence.append(next_note)
            generated.append(next_note)

    return generated



1.

In [5]:
# 1. Подготавливаем данные
X, y = prepare_data_simple(songs, seq_length=20)

# 2. Создаем модель
model = SimpleMusicLSTM(hidden_size=128, num_layers=2)

# 3. Обучаем
model = train_simple_model(
    model, X, y,
    epochs=25,
    batch_size=64,
    lr=0.001
)

# 4. Генерируем музыку
print("\nГенерация музыки...")


seed = songs[0][:10]  # первые 10 нот первой песни

generated_music = generate_music_simple(
    model, seed,
    length=200,
    temperature=0.3  # немного случайности
)

print(f"Сгенерировано {len(generated_music)} нот")


# model, music


Создано 21897 обучающих пар
X shape: (21897, 20, 3), y shape: (21897, 3)
Epoch [5/30], Loss: 16.9391
Epoch [10/30], Loss: 16.8706
Epoch [15/30], Loss: 12.4234
Epoch [20/30], Loss: 12.0933
Epoch [25/30], Loss: 11.9998
Epoch [30/30], Loss: 11.8934
Обучение завершено!

Генерация музыки...
Сгенерировано 200 нот


2. (типа мини эмбединг добавили)

In [7]:
class ImprovedSimpleLSTM(nn.Module):
    """Улучшенная версия вашей модели с минимумом изменений"""

    def __init__(self, hidden_size=128, num_layers=2):
        super().__init__()

        # Добавляем один полносвязный слой перед LSTM
        self.input_projection = nn.Linear(3, 64)

        self.lstm = nn.LSTM(
            input_size=64,  # было 3, стало 64
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )

        self.fc = nn.Linear(hidden_size, 3)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Проекция входных признаков (упрощенный аналог эмбеддингов)
        x_proj = torch.relu(self.input_projection(x))
        x_proj = self.dropout(x_proj)

        lstm_out, _ = self.lstm(x_proj)
        last_output = lstm_out[:, -1, :]
        last_output = self.dropout(last_output)

        prediction = self.fc(last_output)
        return prediction

In [8]:
# 1. Подготавливаем данные
X, y = prepare_data_simple(songs[:500], seq_length=20)

# 2. Создаем модель
model = ImprovedSimpleLSTM(hidden_size=128, num_layers=2)

# 3. Обучаем
model = train_simple_model(
    model, X, y,
    epochs=25,
    batch_size=64,
    lr=0.001
)

# 4. Генерируем музыку
print("\nГенерация музыки...")

# Берем seed из данных
seed = songs[0][:10]  # первые 10 нот первой песни

generated_music = generate_music_simple(
    model, seed,
    length=200,
    temperature=0.4  # немного случайности
)

print(f"Сгенерировано {len(generated_music)} нот")


# model, music


Создано 7427 обучающих пар
X shape: (7427, 20, 3), y shape: (7427, 3)
Epoch [5/25], Loss: 79.0984
Epoch [10/25], Loss: 15.0494
Epoch [15/25], Loss: 15.0115
Epoch [20/25], Loss: 15.0074
Epoch [25/25], Loss: 14.7635
Обучение завершено!

Генерация музыки...
Сгенерировано 200 нот


In [10]:
def tokens_to_midi(tokens, out_midi="markov.mid", resolution=480, step_division=8, program=102):
    step_ticks = resolution // step_division
    music = muspy.Music(resolution=resolution)
    track = muspy.Track(program=program)
    music.tracks.append(track)

    t = 0
    for pitch, dur, rest in tokens:
        t += int(rest) * step_ticks
        track.notes.append(muspy.Note(time=t, pitch=int(pitch), duration=int(dur) * step_ticks, velocity=80))
        t += int(dur) * step_ticks

    muspy.write_midi(out_midi, music)
    return out_midi

tokens_to_midi(generated_music, MODELS + "/models2_2.mid")


'markov2_2.mid'

пока плохо, можно попробовать во время обучения - штрафовать модель за подряд несколько повторяющихся нот (хотя для немецкой музыки это как рах норм)

Зато если наложить аккорды, то может и норм.

или совместить так же генерить аккорды отдельно - и потом совместить с этим.

Как я поняла, соль в том, что в маркоской цепи переход зависит от последней ноту, а здесь от последних 20. (я такую модель потыкала), поэтому так.

### Добавим аккорды

In [14]:
def add_chords_to_melody(melody_tokens, chord_interval=4):
    """
    Добавляет аккорды к мелодии.
    Возвращает список всех нот (мелодия + аккорды) для одного трека.
    """
    all_notes = []

    for i, (pitch, dur, rest) in enumerate(melody_tokens):
        all_notes.append(('melody', pitch, dur, rest))

        # Добавляем аккорд каждые chord_interval нот
        if i % chord_interval == 0:
            # Строим простой мажорный аккорд от ноты мелодии
            base_pitch = int(pitch)

            # (мажорное трезвучие)
            chord_notes = [
                base_pitch,  # основной тон
                base_pitch + 4,  # большая терция
                base_pitch + 7  # чистая квинта
            ]

            # Добавляем каждую ноту аккорда
            for chord_note in chord_notes:
                # Аккорд звучит дольше (в 2 раза дольше ноты мелодии)
                all_notes.append(('chord', chord_note, dur * 3, 0))

    return all_notes


speed_k = 60 # 120 по дефолту

def save_all_in_one_track(all_notes, filename="complete_music.mid"):
    """
    Сохраняет все ноты (мелодию и аккорды) в один трек.
    Мелодия громче, аккорды тише - всё слышно
    """
    music = muspy.Music(resolution=480)
    track = muspy.Track(program=0)  # фортепиано
    music.tracks.append(track)

    t = 0
    for note_type, pitch, dur, rest in all_notes:
        # Учитываем паузу
        t += int(rest) * speed_k

        # Настраиваем громкость: мелодия громче, аккорды тише
        velocity = 80 if note_type == 'melody' else 60

        # Добавляем ноту
        track.notes.append(muspy.Note(
            time=t,
            pitch=int(pitch),
            duration=int(dur) * speed_k,
            velocity=velocity
        ))

        # Для мелодии двигаем время вперед
        # Для аккордов - нет, они звучат параллельно
        if note_type == 'melody':
            t += int(dur) * speed_k

    muspy.write_midi(filename, music)
    print(f"✓ Готово! Сохранено: {filename}")
    print(f"  Всего нот: {len(all_notes)}")
    print(f"  Фортепиано, один трек, всё слышно!")
    return filename


all_notes = add_chords_to_melody(generated_music)
save_all_in_one_track(all_notes, filename=f'{MODELS}/models_akk2_v2.mid')


✓ Готово! Сохранено: models_exp/models_akk2_v2.mid
  Всего нот: 350
  Фортепиано, один трек, всё слышно!


'models_exp/models_akk2_v2.mid'